# BCI-I & IDUN Challenge
## Datset Preparation

This dataset is provided by the BCI-I & IDUN Challenge.
It contains the code used to process the raw data in preparation for the challenge. Participants wishing to use the pre-processed data provided do not need to look at or run this code.
This code is intended as an example for participants that wish to apply pre-processing steps to the raw data prior to separating the data into "trials". 

The dataset includes 1ch EEG recordings from 4 subjects (3 in the training set and 1 in the test set). Sleep-related events were hand-labelled by an expert. 
In this notebook we will:
- Load the raw data
- Pre-process the data
- Cut "epochs" around each sleep-related event
- Save the pre-processed data to file for later use



### Imports

In [75]:

from utils import markers, load_eeg, load_labels, get_event_epochs
from pathlib import Path
import numpy as np


In [76]:
SUBJECT = "S004"  # [S001-3 are in the training set, S004 is in the test set]
data_dir = Path(r'F:\Dropbox (Personal)\BCII\BCI Challenges\202411_IDUN_kaggle_challenge\HACKATHON_Dec2023')

raw_data_dir = data_dir / 'data'
processed_data_dir = data_dir / 'processed'


### Load the raw data

In [77]:
eeg = load_eeg(raw_data_dir, SUBJECT)

Creating RawArray with float64 data, n_channels=1, n_times=3600148
    Range : 0 ... 3600147 =      0.000 ... 14400.588 secs
Ready.
<Info | 7 non-empty values
 bads: []
 ch_names: EEG1
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 1
 projs: []
 sfreq: 250.0 Hz
>


In [78]:
## ANY PREPROCESSING STEPS APPLIED TO EEG SHOUD HAPPEN HERE

In [79]:
labels = load_labels(raw_data_dir, SUBJECT)

labels.head()

,Marker,Timestamp_samples,Epoch
0,REM,10483,2
1,REM,11543,2
2,REM,12792,2
3,REM,14027,2
4,REM,15239,3


### Create epochs around each event


In [80]:
epoch_eeg_data = []
epoch_labels = []

for marker in markers:
    epochs = get_event_epochs(eeg, labels, marker)
    if epochs is None:
        print(f"!!! --- No epochs found for marker {marker}")
        continue

    _data = epochs.get_data()[:, 0, :]
    epoch_eeg_data.append(_data)
    epoch_labels.append(np.full(len(_data), marker))

assert len(epoch_eeg_data) == len(epoch_labels)


Not setting metadata
183 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 183 events and 7501 original time points ...
0 bad epochs dropped
Not setting metadata
197 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 197 events and 7501 original time points ...
0 bad epochs dropped
Not setting metadata
56 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 56 events and 7501 original time points ...
0 bad epochs dropped
Not setting metadata
5 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 5 events and 7501 original time points ...
0 bad epochs dropped
Not setting metadata
4 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 4 events and 7501 original time points 

c:\Users\feder\Documents\github\BCII-IDUN-CHallenge\utils.py:65: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  
c:\Users\feder\Documents\github\BCII-IDUN-CHallenge\utils.py:65: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  
c:\Users\feder\Documents\github\BCII-IDUN-CHallenge\utils.py:65: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  


In [81]:
X = np.concatenate(epoch_eeg_data, axis=0)
y = np.concatenate(epoch_labels, axis=0)

print(X.shape, y.shape)

np.save(processed_data_dir / f"{SUBJECT}_X.npy", X)
np.save(processed_data_dir / f"{SUBJECT}_y.npy", y)



(448, 7501) (448,)
